<a href="https://colab.research.google.com/github/andouglasjr/PDS_IF_PAAS/blob/main/Arquitetura_MVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
# Instituto Federal do Rio Grande do Norte
# Campus Avançado de Parelhas
# Projeto de Software
# Professor: Andouglas Gonçalves da Silva Júnior
```



##Arquitetura MVC

Neste notebook vamos implementar um simples exemplo estruturado na arquitetura Model-View-Controller (MVC). 

Nota: Aqui é um simples exemplo, apenas para entender como essa arquitetura pode ser implementada. Por isso, não vamos usar interfaces mais elaboradas, nem banco de dados.

Primeiro, faça os imports das bibliotecas que vamos utilizar neste projeto:

1 - Pandas - Auxilia no processamento de dados;


In [1]:
import pandas as pd

###CRUD

De qualquer forma, precisamos de uma maneira de "armazenar" dados para que possam ser acessados pelo sistema por meio do modelo. Para tal, vamos usar uma simples lista em python que irá armazenar as informações desejadas e vamos visualizar essa lista em formato de tabela utilizando a biblioteca Pandas.

In [4]:
ifsDoSerido = [
  {'sigla':'PAAS', 'cidade': 'Parelhas'},
  {'sigla':'CN', 'cidade': 'Currais Novos'},
  {'sigla':'CA', 'cidade': 'Caicó'}
]

tabela = pd.DataFrame(ifsDoSerido)

In [5]:
tabela

,sigla,cidade
0,PAAS,Parelhas
1,CN,Currais Novos
2,CA,Caicó


Vamos criar uma classe que irá criar, ler, atualizar e apagar dados nessa tabela, como se fosse o nosso banco de dados.

In [22]:
items = list() #Variável global onde nossos dados serão armazenados

#As duas próximas funções correspondem a função Create do CRUD

def create_items(app_items): #criar a partir de uma lista
  global items
  items = app_items

def create_item(sigla, cidade): #criar apenas um item
  global items
  items.append({'sigla':sigla, 'cidade':cidade})

#As duas próximas funções correspondem a função Read do CRUD

def read_item(sigla): #ler apenas um item a partir da sigla
  global items
  myitems = list(filter(lambda x:x['sigla'] == sigla, items))
  return myitems[0]

def read_items(): #ler todos os itens da lista
  global items
  return [item for item in items]

#A próxima função corresponde a função UPDATE do CRUD

def update_item(sigla, cidade): #atualiza um deterinado item pela sigla
  global items
  idxs_items = list(
        filter(lambda i_x: i_x[1]['sigla'] == sigla, enumerate(items)))
  if idxs_items:
        i, item_to_update = idxs_items[0][0], idxs_items[0][1]
        items[i] = {'sigla': sigla, 'cidade': cidade}
  
#A próxima função corresonde a função DELETE do CRUD

def delete_item(sigla): #deleta um deterinado item pela sigla
  global items
  idxs_items = list(
        filter(lambda i_x: i_x[1]['sigla'] == sigla, enumerate(items)))
  if idxs_items:
        i, item_to_delete = idxs_items[0][0], idxs_items[0][1]
        del items[i]



##Model

Agora que temos o nosso "banco de dados" vamos acessa-lo por meio do modelo. Basicamente, acessaremos as funções criadas anteriormente sempre por meio desse modelo.

In [23]:
class Model(object):

  def __init__(self, application_items):
    self._item_type = 'instituto'
    self.create_items(application_items)
  
  @property
  def item_type(self):
    return self._item_type
  
  @item_type.setter
  def item_type(self, new_item_type):
    self._item_type = new_item_type
  
  def create_item(self, sigla, cidade):
    create_item(sigla, cidade)
  
  def create_items(self, items):
    create_items(items)
  
  def read_item(self, sigla):
    return read_item(sigla)
  
  def read_items(self):
    return read_items()
  
  def update_item(self, sigla, cidade):
    update_item(sigla, cidade)
  
  def delete_item(self, sigla):
    delete_item(sigla) 

###View

Agora vamos criar a "interface" com o usuário. O importante aqui é notar que não existe lógica nessa parte do código, apenas métodos estáticos que vão apresentar alguma coisa ao usuário. Claro que em uma aplicação real, a interface não seria a saída do terminal, mas aqui é apenas para fins de entender os conceitos relacionados ao MVC.

In [44]:
class View(object):

  @staticmethod
  def show_item(item_type, item_sigla, item):
    print('Sigla: {} - Cidade: {}'.format(item_sigla, item['cidade']))
    
  
  @staticmethod
  def display_item_stored(item, item_type):
    print('O instituto {} foi adicionado com sucesso na nossa lista'.format(item))
  
  @staticmethod
  def display_item_updated(item, o_cidade, n_cidade):
    print('Atualizando o item {} da cidade antiga {} para {}.'.format(item, o_cidade, n_cidade))
  
  @staticmethod
  def display_item_deleted(sigla):
    print('O item com a sigla {} foi removido'.format(sigla))


###Controller

Agora que temos a Interface e o modelo criados podemos interliga-los por meio do contrlador. Note que vamos chamar as classes View e Model no init da classe Controller para fazer essa interligação.

In [25]:
class Controller(object):

  def __init__(self, model, view):
    self.model = model
    self.view = view
  
  def show_item(self, item_sigla):
    item = self.model.read_item(item_sigla)
    item_type = self.model.item_type
    self.view.show_item(item_type, item_sigla, item)
  
  def insert_item(self, sigla, cidade):
    item_type = self.model.item_type
    self.model.create_item(sigla, cidade)
    self.view.display_item_stored(sigla, item_type)
  
  def update_item(self, sigla, cidade):
    item_type = self.model.item_type

    older = self.model.read_item(sigla)
    self.model.update_item(sigla, cidade)
    self.view.display_item_updated(sigla, older['sigla'],cidade)



###Testando nossa arquitetura

Vamos testar se nossa arquitetura está funcionando como o esperado com as funções já implementadas.

In [45]:
#Para inicializar nosso "banco" vamos utilizar a seguinte lista

ifsDoSerido = [
  {'sigla':'PAAS', 'cidade': 'Parelhas'},
  {'sigla':'CN', 'cidade': 'Currais Novos'}
]

#Incializando a aplicação

c = Controller(Model(ifsDoSerido), View())



In [46]:
c.show_item('PAAS')

Sigla: PAAS - Cidade: Parelhas


In [47]:
c.insert_item('CA', 'Caicó')

O instituto CA foi adicionado com sucesso na nossa lista


### Exercícios

1. Adicione uma função que apresente todos os elementos da lista.
2. Altere a função anterior para aparesentar os elementos em formato de tablea usando o Pandas.
3. Adicione uma função que conte quantos elementos tem na lista.